In [1]:
using Revise

using CellBasedModels
using CUDA

using Accessors
using BenchmarkTools

using SparseArrays

In [36]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

)

com1 = Community(model, N = 3)
com1.xx .= 3 
com1.x .= 10
setfield!(com1, :id, 4:8)


com2 = Community(model, N = 5)
com2.xx .= 4

function append(com1::Community, com2::Community)

    com = Community(com1.abm, N = com1.N+com2.N)

    for (i,j) in com1.abm.parameters
        if j.scope == :agent
            com[i] .= [com1[i];com2[i]]
        end
    end

    idMax = maximum(com1.id)+1
    newIdMax = idMax+1+com2.N
    id =  Int.([com1.id; idMax:newIdMax])

    setfield!(com, :id, id)

    return com

end

com = append(com1,com2)
for (i,j) in com.abm.parameters
    println(i, " ", com[i])
end

loadToPlatform!(com)
println(com.id)
com.idMax_

nothing
x [10.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0]
y [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
z [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
xx [3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0]
[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]


Base.Threads.Atomic{Int64}(8)

In [27]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

    agentODE = quote
        
        dt(x) = 1

    end,

    agentRule = quote
        @addAgent()
    end

)

com = Community(model, dt=0.1, N=(Main=5,))
com.x = 2
com.y = 3

# com_original = deepcopy(com)
loadToPlatform!(com, preallocateAgents=(Main=2,))
# println(prettify(com.abm.declaredUpdatesCode[:agentODE]))
# step!(com)

# model2 = ABM(3,

#     baseModelInit=[model],

#     agent=Agent(:sphere,id=:idSphere,pos=(xSphere=Float64,ySphere=Float64,zSphere=Float64),parameters=(radius=Float64,)),

#     agentODE = quote
        
#         @inagent sphere begin
#             dt(radius) = 1
#         end

#         @inagent Main begin
#             dt(y) = 9
#             z += 1
#         end
#     end
# )

# com = Community(model2, dt=0.1, N=(Main=5,sphere=3))
# com.radius = 1
# com.x = 2
# com.y = 3

# # com_original = deepcopy(com)
# loadToPlatform!(com, preallocateAgents=(Main=2,sphere=1))
# step!(com)
# # bringFromPlatform!(com)

# prettify(model2.declaredUpdatesCode[:agentODE])

for (u,j) in pairs(com.parameters)
    println(u, "\t",j)
end

LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Float64}[39m[0m to an object of type [91mInt64[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::ColorTypes.Gray[39m) where T<:Real
[0m[90m   @[39m [32mColorTypes[39m [90m~/.julia/packages/ColorTypes/vpFgh/src/[39m[90m[4mconversions.jl:113[24m[39m
[0m  convert(::Type{T}, [91m::ColorTypes.Gray24[39m) where T<:Real
[0m[90m   @[39m [32mColorTypes[39m [90m~/.julia/packages/ColorTypes/vpFgh/src/[39m[90m[4mconversions.jl:114[24m[39m
[0m  convert(::Type{T}, [91m::DualNumbers.Dual[39m) where T<:Union{Real, Complex}
[0m[90m   @[39m [36mDualNumbers[39m [90m~/.julia/packages/DualNumbers/5knFX/src/[39m[90m[4mdual.jl:24[24m[39m
[0m  ...


In [34]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

    agentODE = quote
        
        dt(x) = 1

    end,

    platform=GPU()

)

model2 = ABM(3,

    baseModelInit=[model],

    agent=Agent(:sphere,id=:idSphere,pos=(xSphere=Float64,ySphere=Float64,zSphere=Float64),parameters=(radius=Float64,)),

    agentODE = quote
        
        @inagent sphere begin
            dt(radius) = 1
        end

        @inagent Main begin
            dt(y) = 9
        end
    end
)

com = Community(model2, dt=0.1, N=(Main=20,sphere=10))
com.radius = 1
com.x = 2
com.y = 3

com_original = deepcopy(com)
loadToPlatform!(com, preallocateAgents=(Main=2,sphere=1))
bringFromPlatform!(com)

l = []
for sym in keys(com.parameters)
    k = all(com_original[sym] == com[sym])
    if !k
        if typeof(com_original[sym]) <: Threads.Atomic && typeof(com[sym]) <: Threads.Atomic
            if com_original[sym][] == com[sym][]
                k = true
            end
        end
    end
end
all(l)

true

In [33]:
Threads.Atomic{Int}(0) == Threads.Atomic{Int}(0)

false

In [91]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

    agentRule = quote

        x = 1

        @removeAgent()
        @addAgent(
            x = 1,
            xx = 0
        )

    end,

    platform=CPU()

)

com = Community(model, N=1)

Community.


In [96]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

    agentRule = quote

        x = 1

        @removeAgent()
        @addAgent(
            x = 1,
            xx = 0
        )

    end,

    agentODE = quote
        dt(y) = -1
        dt(x) = -1
    end,

    platform=CPU()

)
prettify(model.declaredUpdatesCode[:agentRule])
prettify(model.declaredUpdates[:agentRule])

[(i,j.pos) for (i,j) in model.parameters]

24-element Vector{Tuple{Symbol, Int64}}:
 (:t, 0)
 (:dt, 0)
 (:NMedium, 0)
 (:simBox, 0)
 (:dx, 0)
 (:dy, 0)
 (:dz, 0)
 (:xₘ, 0)
 (:yₘ, 0)
 (:zₘ, 0)
 (:x, 1)
 (:y, 2)
 (:z, 0)
 (:id, 0)
 (:xx, 0)
 (:Main_N_, 0)
 (:Main_NMax_, 0)
 (:Main_idMax_, 0)
 (:Main_NAdd_, 0)
 (:Main_NRemove_, 0)
 (:Main_NSurvive_, 0)
 (:Main_flagSurvive_, 0)
 (:Main_holeFromRemoveAt_, 0)
 (:Main_repositionAgentInPos_, 0)

In [89]:
model = ABM(3,

    agent=Dict(:xx=>Float64),

    agentRule = quote
        
        x += 1

    end,

    platform=GPU()

)

model2 = ABM(3,

    baseModelInit=[model],

    agentRule = quote
        
        x += 1

    end
)
prettify(model2.declaredUpdatesCode[:agentRule])
prettify(model2.declaredUpdates[:agentRule])

:(@inagent Main begin
          x__ += 1
          x__ += 1
      end)

In [96]:
model = ABM(3,

    agent=[
            Agent(:hola, 
                pos = (x=Float64,y=Float64,z=Float64),
                id = :id,
                parameters=Dict(:xx=>Float64)
            ), 
        ],

    agentRule = quote
        
        x += 1

    end,

    platform=GPU()

)

model2 = ABM(3,

    baseModelInit=[model],

    # agent=[
    #         Agent(:hola2, 
    #             pos = (x1=Float64,y1=Float64,z1=Float64),
    #             id = :id1,
    #             parameters=Dict(:xx1=>Float64)
    #         ), 
    #     ],

    agentRule = quote
        
        x += 1

    end
)
prettify(model2.declaredUpdatesCode[:agentRule])
# prettify(model2.declaredUpdates[:agentRule])

:(function (community,)
      function rule_(t, dt, NMedium, simBox, dx, dy, dz, NAdd_, NRemove_, NSurvive_, flagSurvive_, holeFromRemoveAt_, repositionAgentInPos_, x, y, z, id, xx, hola_N_, hola_NMax_, hola_idMax_, x__)
          @inbounds Threads.@threads(for i1_ = 1:1:hola_N_[1]
                      x__[i1_] += 1
                  end)
          @inbounds Threads.@threads(for i1_ = 1:1:Main_N_[1]
                      x__[i1_] += 1
                  end)
          return
      end
      rule_(community.t, community.dt, community.NMedium, community.simBox, community.dx, community.dy, community.dz, community.NAdd_, community.NRemove_, community.NSurvive_, community.flagSurvive_, community.holeFromRemoveAt_, community.repositionAgentInPos_, community.parameters[Symbol("x")], community.parameters[Symbol("y")], community.parameters[Symbol("z")], community.parameters[Symbol("id")], community.parameters[Symbol("xx")], community.parameters[Symbol("hola_N_")], community.parameters[Symbol("h

In [68]:
model = ABM(3,

    agent=[
            Agent(:hola, 
                pos = (x=Float64,y=Float64,z=Float64),
                id = :id,
                parameters=Dict(:xx=>Float64)
            ), 
        ],

    agentRule = quote
        
        x += 1

    end,

    platform=GPU()

)
prettify(model.declaredUpdatesCode[:agentRule])
prettify(model.declaredUpdates[:agentRule])

h9li

:(@inagent hola x__ += 1)

In [64]:
model = ABM(3,

    agent=[
            Agent(:hola, 
                pos = (x=Float64,y=Float64,z=Float64),
                id = :id,
                parameters=Dict(:xx=>Float64)
            ), 
            Agent(:hola2, 
                pos=(x2=Float64,y2=Float64,z2=Float64),
                id=:id2,
                parameters=Dict(:xx2=>Float64)
            )
        ],

    agentRule = quote
        
        @inagent hola begin
            x += 1
        end
        @inagent hola2 begin
            x += 1
        end
        @inagent hola2 begin
            x += 1
        end
        # x+1

    end,

    platform=GPU()

)
prettify(model.declaredUpdatesCode[:agentRule])
prettify(model.declaredUpdates[:agentRule])

quote
    @inagent hola x__ += 1
    @inagent hola2 begin
            x__ += 1
            x__ += 1
        end
end

In [28]:
function collect_variables(expr, variables=Set{Symbol}())
    if expr isa Expr
        if expr.head == :call || expr.head == :macrocall
            # Traverse function/macro calls to extract variables
            for arg in expr.args
                collect_variables(arg, variables)
            end
        elseif expr.head == :block || expr.head == :if || expr.head == :while || expr.head == :for
            # Traverse block structures (like if, while, for, begin)
            for arg in expr.args
                collect_variables(arg, variables)
            end
        elseif expr.head == :assign
            # For assignments, collect the left-hand side as a variable
            if expr.args[1] isa Symbol
                push!(variables, expr.args[1])
            end
            # Recursively process the right-hand side of the assignment
            collect_variables(expr.args[2], variables)
        else
            # For other expressions, recursively process arguments
            for arg in expr.args
                collect_variables(arg, variables)
            end
        end
    elseif expr isa Symbol
        # Collect the symbol if it is a variable
        push!(variables, expr)
    end
    return variables
end

collect_variables (generic function with 2 methods)

In [38]:
m = quote
    a = 0
    b = 4
    if z == 4
        @loopOverNeighbors d
    end
    cos(x)
end

[i for i in collect_variables(m) if !(i in names(Base)) && !(i in names(CellBasedModels))]

5-element Vector{Symbol}:
 :a
 :b
 :d
 :z
 :x